In [14]:
import pandas as pd
import numpy as np


In [15]:
data_folder = './dataset/'
def merge_forcage_piezo(forcage_file, merge_file ='static_attributes/piezo_characs.csv', output_file = None):
    df_forcage =  pd.read_csv(data_folder + 'time_series/forçages/' + forcage_file + '.txt', 
                  sep=';', 
                  skiprows=85,
                  comment='#')
    df_forcage.columns = df_forcage.columns.str.strip()
    df_piezo_charachs =  pd.read_csv(data_folder + merge_file,sep=';',encoding='latin-1')
    
    bss_id = df_piezo_charachs[df_piezo_charachs['limnit'] == forcage_file ]['BSS_ID'].values[0]
    
    df_piezo =  pd.read_csv(data_folder + "time_series/piezos/"+bss_id+".csv")
    
    df_piezo.head()

    df_piezo["date_mesure"] = (
        pd.to_datetime(df_piezo["date_mesure"])
        .dt.strftime("%Y%m%d")
        .astype("Int64")
    )

    df_piezo['Date'] = df_piezo['date_mesure'].astype('int64')
    # garder que les colonnes 'date_mesure' et 'niveau_nappe_eau'
    df_piezo = df_piezo[['Date', 'niveau_nappe_eau']]
    # join df_forcage et df_piezo sur la colonne 'date'
    df_merged = pd.merge(df_forcage, df_piezo, on='Date', how='inner')

    df_merged.to_csv(data_folder + '/preprocessed/' + (output_file or forcage_file) + '.csv', index=False)
    
merge_forcage_piezo('A146020302','static_attributes/piezo_characs.csv')
    
   